In [34]:
from pathlib import Path
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import soundfile as sf
import pandas as pd

import os 
import itertools
import re
import math
import csv
import random
from sklearn import tree
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn import linear_model
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier
from scipy.fftpack import fftn, ifftn, fft, ifft
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from scipy import signal
import pickle
from sklearn.datasets import make_classification
from sklearn.externals import joblib
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import ShuffleSplit
import warnings

warnings.filterwarnings('ignore') 
#df = pd.read_csv("audible/Features10too200hz_aud.csv")
#df = pd.read_csv("audible/unfilteredFeatures_aud.csv")
#df = pd.read_csv("../featuress/allEmgData10too200hz_withAudible.csv")
#df = pd.read_csv("../featuress/allEmgData10too200hz_withAudible_6Channel.csv")
#df = pd.read_csv("../featuress/allEmgData10to200hz_withAudible.csv")
df = pd.read_csv("../featuress/allEmgData_withAudible.csv")
#df = pd.read_csv("../featuress/allEMGData10too200hz_withAudible_window.csv")




#dfs = df[(df.modeID != 'audible')]
df

,channel_1_mav,channel_1_mav2,channel_1_rms,channel_1_iav,channel_1_ssi,channel_1_var,channel_1_wl,channel_1_iemg,channel_1_aac,channel_1_zc,...,channel_7_aac,channel_7_zc,channel_7_ssc,channel_7_wamp,channel_7_medf,channel_7_meanf,userID,sessionID,uttID,modeID
0,0.067106,0.067103,0.081097,224.738037,22.025275,0.003286,191.597839,224.738037,0.057210,682,...,7.289956e-08,0,0,0,0.0,0.014719,1,1001,10010001,audible
1,0.063718,0.063648,0.075014,143.621399,12.683361,0.002283,103.957947,143.621399,0.046122,288,...,1.083144e-07,0,0,0,0.0,0.014499,1,1001,10010002,audible
2,0.067169,0.067090,0.081422,272.503754,26.896239,0.003316,227.992035,272.503754,0.056197,816,...,6.017763e-08,0,0,0,0.0,0.015419,1,1001,10010003,audible
3,0.065079,0.065007,0.078301,271.053162,25.535573,0.002825,224.132660,271.053162,0.053813,772,...,5.861720e-08,0,0,0,0.0,0.015302,1,1001,10010004,audible
4,0.065251,0.065301,0.077894,306.745789,28.522983,0.002760,239.254150,306.745789,0.050894,756,...,5.193376e-08,0,0,0,0.0,0.014725,1,1001,10010005,audible
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7374,0.071839,0.071598,0.085800,134.123810,13.744313,0.003407,102.125336,134.123810,0.054700,296,...,1.307663e-07,0,0,0,0.0,0.015357,8,8101,81010506,audible
7375,0.071289,0.070963,0.084742,139.227325,14.024961,0.003161,94.420532,139.227325,0.048346,292,...,1.250080e-07,0,0,0,0.0,0.015307,8,8101,81010507,audible
7376,0.072052,0.072028,0.085698,122.127991,12.448364,0.003418,85.883698,122.127991,0.050669,250,...,1.440358e-07,0,0,0,0.0,0.015211,8,8101,81010508,audible
7377,0.072907,0.072573,0.087714,142.459381,15.033598,0.003763,105.453644,142.459381,0.053968,312,...,1.249440e-07,0,0,0,0.0,0.014722,8,8101,81010509,audible


In [31]:
def split_data_per_session(session_id):
    temp_df = df[(df.sessionID == session_id)]
    data = temp_df.copy()
    data = shuffle(data)
    y = data['modeID']
    data.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return train_test_split(data, y, test_size=0.2)

def split_data_session(train_user_ids,test_user_ids):
    temp_df = df
    data = temp_df.copy()
    train_df = data.loc[data['sessionID'].isin(train_user_ids)]
    test_df = data.loc[data['sessionID'].isin(test_user_ids)]
    X_train = train_df.copy()
    X_test = test_df.copy()
    y_train = X_train['userID']
    y_test = X_test['userID']
    X_train.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    X_test.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return X_train, X_test, y_train, y_test 

def split_data_utterance(train_user_ids,test_user_ids):
    temp_df = df
    data = temp_df.copy()
    train_df = data.loc[data['userID'].isin(train_user_ids)]
    test_df = data.loc[data['userID'].isin(test_user_ids)]
    X_train = train_df.copy()
    X_test = test_df.copy()
    y_train = X_train['userID']
    y_test = X_test['userID']
    X_train.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    X_test.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return X_train, X_test, y_train, y_test 

def split_data(train_user_ids,test_user_ids):
    temp_df = df
    data = temp_df.copy()
    train_df = data.loc[data['userID'].isin(train_user_ids)]
    test_df = data.loc[data['userID'].isin(test_user_ids)]
    X_train = train_df.copy()
    X_test = test_df.copy()
    y_train = X_train['modeID']
    y_test = X_test['modeID']
    X_train.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    X_test.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return X_train, X_test, y_train, y_test 

In [32]:
def lda():
    ids = [1001,1002,1003,2001,2002,2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,2011, 2012, 2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,
        2023,2024,2025,2026,2027,2028,2029, 2030,2031, 2032,2101,4001,4002,7001,7002,8001,8002,8003,8004,8005,8006,8007,8008,8009, 8010, 8011,8012,8013,8014,8015, 8016, 8017, 8018,8019, 8101]
    scores = np.array([])
    for x in range (len(ids)):
        test_ids = []
        test_ids = ids[x:x+1]
        train_ids = np.delete(ids,x)
        #print (test_ids)
        X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
        k_clf = LinearDiscriminantAnalysis()
        k_clf.fit(X_train, y_train)
        score = k_clf.score(X_test, y_test)
        sscore = np.array([score])
        scores = np.concatenate([scores,sscore],axis=0)
        print("Accuracy for session " + str(test_ids) +" is "+str(score))
    print ("Durchschnitt: ")
    print((scores.mean()))
    print(scores.std())

In [24]:
lda()

Accuracy for session [1001] is 0.68
Accuracy for session [1002] is 0.8333333333333334
Accuracy for session [1003] is 0.96
Accuracy for session [2001] is 0.9933333333333333
Accuracy for session [2002] is 1.0
Accuracy for session [2003] is 1.0
Accuracy for session [2004] is 1.0
Accuracy for session [2005] is 1.0
Accuracy for session [2006] is 1.0
Accuracy for session [2007] is 1.0
Accuracy for session [2008] is 1.0
Accuracy for session [2009] is 1.0
Accuracy for session [2010] is 1.0
Accuracy for session [2011] is 1.0
Accuracy for session [2012] is 1.0
Accuracy for session [2013] is 1.0
Accuracy for session [2014] is 1.0
Accuracy for session [2015] is 0.98
Accuracy for session [2016] is 1.0
Accuracy for session [2017] is 1.0
Accuracy for session [2018] is 1.0
Accuracy for session [2019] is 1.0
Accuracy for session [2020] is 1.0
Accuracy for session [2021] is 1.0
Accuracy for session [2022] is 1.0
Accuracy for session [2023] is 1.0
Accuracy for session [2024] is 1.0
Accuracy for session [

In [29]:
lda()

Accuracy for session [1001] is 0.6666666666666666
Accuracy for session [1002] is 0.8333333333333334
Accuracy for session [1003] is 0.9466666666666667
Accuracy for session [2001] is 0.9866666666666667
Accuracy for session [2002] is 0.98
Accuracy for session [2003] is 1.0
Accuracy for session [2004] is 1.0
Accuracy for session [2005] is 1.0
Accuracy for session [2006] is 1.0
Accuracy for session [2007] is 1.0
Accuracy for session [2008] is 1.0
Accuracy for session [2009] is 1.0
Accuracy for session [2010] is 1.0
Accuracy for session [2011] is 1.0
Accuracy for session [2012] is 1.0
Accuracy for session [2013] is 1.0
Accuracy for session [2014] is 1.0
Accuracy for session [2015] is 0.98
Accuracy for session [2016] is 1.0
Accuracy for session [2017] is 1.0
Accuracy for session [2018] is 1.0
Accuracy for session [2019] is 1.0
Accuracy for session [2020] is 1.0
Accuracy for session [2021] is 1.0
Accuracy for session [2022] is 1.0
Accuracy for session [2023] is 1.0
Accuracy for session [2024] 

In [33]:
lda() #ungefilterter ohne 5

Accuracy for session [1001] is 0.9533333333333334
Accuracy for session [1002] is 0.9933333333333333
Accuracy for session [1003] is 0.9933333333333333
Accuracy for session [2001] is 0.9733333333333334
Accuracy for session [2002] is 1.0
Accuracy for session [2003] is 1.0
Accuracy for session [2004] is 0.98
Accuracy for session [2005] is 0.9933333333333333
Accuracy for session [2006] is 1.0
Accuracy for session [2007] is 0.9933333333333333
Accuracy for session [2008] is 0.98
Accuracy for session [2009] is 1.0
Accuracy for session [2010] is 1.0
Accuracy for session [2011] is 0.98
Accuracy for session [2012] is 1.0
Accuracy for session [2013] is 1.0
Accuracy for session [2014] is 1.0
Accuracy for session [2015] is 1.0
Accuracy for session [2016] is 0.98
Accuracy for session [2017] is 0.98
Accuracy for session [2018] is 1.0
Accuracy for session [2019] is 1.0
Accuracy for session [2020] is 1.0
Accuracy for session [2021] is 1.0
Accuracy for session [2022] is 1.0
Accuracy for session [2023] is

In [36]:
lda() #ungefiltert mit 5

Accuracy for session [1001] is 0.96
Accuracy for session [1002] is 0.9866666666666667
Accuracy for session [1003] is 0.9866666666666667
Accuracy for session [2001] is 0.9866666666666667
Accuracy for session [2002] is 1.0
Accuracy for session [2003] is 1.0
Accuracy for session [2004] is 0.9666666666666667
Accuracy for session [2005] is 1.0
Accuracy for session [2006] is 1.0
Accuracy for session [2007] is 1.0
Accuracy for session [2008] is 1.0
Accuracy for session [2009] is 1.0
Accuracy for session [2010] is 1.0
Accuracy for session [2011] is 1.0
Accuracy for session [2012] is 1.0
Accuracy for session [2013] is 1.0
Accuracy for session [2014] is 1.0
Accuracy for session [2015] is 1.0
Accuracy for session [2016] is 0.98
Accuracy for session [2017] is 1.0
Accuracy for session [2018] is 1.0
Accuracy for session [2019] is 1.0
Accuracy for session [2020] is 1.0
Accuracy for session [2021] is 1.0
Accuracy for session [2022] is 1.0
Accuracy for session [2023] is 1.0
Accuracy for session [2024] 